In [70]:
import pandas as pd
import os

# formating data

In [52]:
df = pd.read_csv('/Users/tajo5912/anova_down/forANOVA.csv',
                sep=',')
df['cond_rep'] = df['Condition'] + ';' + df['BioRep']

cy_list = list(set(df['Cytokine']))
cy_list.sort()
len(cy_list)

105

In [53]:
minidf = df[df['Cytokine'] == cy_list[0]]
tech_rep_list = list(set(df['cond_rep']))
len(tech_rep_list)
tech_rep_list

['T21_CA;R1',
 'D21_Control;R1',
 'D21_CA;R2',
 'D21_IFN_CA;R2',
 'T21_IFN_CA;R2',
 'D21_Control;R2',
 'T21_IFN;R1',
 'T21_Control;R2',
 'T21_IFN_CA;R1',
 'D21_IFN_CA;R1',
 'D21_IFN;R1',
 'T21_CA;R2',
 'D21_IFN;R2',
 'T21_IFN;R2',
 'T21_Control;R1',
 'D21_CA;R1']

In [54]:
av_list = []
for cy in cy_list:
    minidf = df[df['Cytokine'] == cy]
    for tech in tech_rep_list:
        minitech = minidf[minidf['cond_rep'] == tech]['Value'].sum()/2
        av_list.append([cy,tech,minitech])

In [55]:
avdf=pd.DataFrame.from_records(av_list)
avdf.columns = ['cytokine', 'cond_rep', 'value']
len(df) == len(avdf)*2

True

In [60]:
avdf['sample'] = avdf['cond_rep'].str.split(';',expand=True)[0]

In [61]:
avdf.sort_values(by=['cytokine','sample']).to_csv('/Users/tajo5912/anova_down/av_values_for_anova.txt',
           sep='\t', index=False)

In [64]:
avdf['sample'].unique()

dd={'T21_CA':'G', 
    'D21_Control':'A', 'D21_CA':'C', 'D21_IFN_CA':'D', 'T21_IFN_CA':'H',
       'T21_IFN':'F', 'T21_Control':'E', 'D21_IFN':'B'}

In [66]:
avdf['sample_group'] = avdf['sample']
avdf=avdf.replace({"sample_group": dd})

In [68]:
avdf.sort_values(by=['cytokine','sample_group']).to_csv('/Users/tajo5912/anova_down/av_values_for_anova.txt',
           sep='\t', index=False)

# submitting jobs

In [74]:
script='/Users/tajo5912/anova_down/submit_anova.sbatch'
samples = list(set(avdf['sample']))
for sample in samples:
    for cy in cy_list:
        os.system('sbatch ' + script + ' ' + cy + ' ' + sample)
#     os.system('mkdir ' + '/Users/tajo5912/anova_down/' + sample)

# cat jobs

In [84]:
# open the sample file used
for sample in samples:
    bigdf=pd.DataFrame()
    for cy in cy_list:
        file = open('/Users/tajo5912/anova_down/'+sample+'/'+cy+'.txt')
        content = file.readlines()
        ll=[]
        for i in range(len(content)):
            if 'Intercept' in content[i]:
                ll.append(i)
            if '---' in content[i]:
                ll.append(i)
        try:
            content=content[ll[0]:ll[1]]
        except:
            ll=[]
            for i in range(len(content)):
                if 'Intercept' in content[i]:
                    ll.append(i)
                if 'freedom' in content[i]:
                    ll.append(i-1)
            content=content[ll[0]:ll[1]]
        clo=[]
        for i in content:
            i=i.replace("sample", "")
            i=i.replace('*', '')
            i=" ".join(i.split())
            i=i.replace(' ', ',')
            i=i.split(',')
            clo.append(i)
        clo = pd.DataFrame.from_records(clo)
        clo['cytokine'] =cy
        bigdf=pd.concat([bigdf,clo])
    bigdf.to_csv('/Users/tajo5912/anova_down/'+ sample + '_combined_output.txt', sep='\t', index=False)